<a href="https://colab.research.google.com/github/Anushka-2906/MARS_2025/blob/main/MARS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install flask flask-ngrok python-docx pytesseract Pillow pymupdf langdetect
!apt install tesseract-ocr -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [2]:
import os
import fitz  # PyMuPDF for PDFs
import docx
import pytesseract
from PIL import Image, ImageOps, ImageEnhance, ImageFilter
from werkzeug.utils import secure_filename
from flask import Flask, request, render_template, jsonify, send_file
import re
from langdetect import detect, DetectorFactory
from datetime import datetime
import mimetypes
from collections import Counter
import json

In [3]:
DetectorFactory.seed = 0

In [4]:
pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"

In [5]:
app = Flask(__name__)
UPLOAD_FOLDER = 'uploads'
OUTPUT_FOLDER = 'output'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['OUTPUT_FOLDER'] = OUTPUT_FOLDER

In [6]:
# Utility: Clean text

In [7]:
def clean_text(text):
    return re.sub(r'\s+', ' ', text.strip())

In [8]:
# Extract text based on file type

In [9]:
def extract_text_pdf(file_path):
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

In [10]:
def extract_text_docx(file_path):
    doc = docx.Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

In [11]:
def extract_text_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

In [12]:
def extract_text_image(file_path):
    image = Image.open(file_path)
    image = image.resize((image.width * 2, image.height * 2), Image.LANCZOS)
    gray = ImageOps.grayscale(image)
    sharp = gray.filter(ImageFilter.SHARPEN)
    contrast = ImageEnhance.Contrast(sharp).enhance(3)
    return pytesseract.image_to_string(contrast, config='--psm 4')

In [13]:
# Semantic content: enhanced for image-based inputs

In [14]:
def extract_semantic_content(text):
    lines = text.split('\n')
    clean_lines = [line.strip() for line in lines if line.strip()]
    if len(clean_lines) == 0:
        return ""
    elif len(clean_lines) == 1:
        return clean_lines[0]
    elif len(clean_lines) == 2:
        return clean_lines[0] + " " + clean_lines[1]
    else:
        return " ".join(clean_lines[:min(6, len(clean_lines))])

In [15]:
# Keyword extractor (frequency-based)

In [16]:
def extract_keywords(text, top_n=10):
    words = re.findall(r'\b[a-zA-Z]{5,}\b', text.lower())
    freq = Counter(words)
    return [word for word, _ in freq.most_common(top_n)]

In [17]:
# Metadata Generator

In [18]:
def generate_metadata(text, filename):
    lines = text.split('\n')
    clean_lines = [line.strip() for line in lines if line.strip()]
    title = clean_lines[0] if clean_lines else "Untitled"
    word_count = len(re.findall(r'\w+', text))
    keywords = extract_keywords(text)
    semantic_summary = extract_semantic_content(text)
    try:
        language = detect(text) if len(text) > 20 else "unknown"
    except:
        language = "unknown"
    created_time = datetime.now().isoformat()
    file_type, _ = mimetypes.guess_type(filename)

In [30]:
import os
import mimetypes
import re
from datetime import datetime
from langdetect import detect

def generate_metadata(text, filename):
    lines = text.split('\n')
    clean_lines = [line.strip() for line in lines if line.strip()]
    title = clean_lines[0] if clean_lines else "Untitled"
    word_count = len(re.findall(r'\w+', text))
    keywords = extract_keywords(text)
    semantic_summary = extract_semantic_content(text)

    try:
        language = detect(text) if len(text) > 20 else "unknown"
    except:
        language = "unknown"

    created_time = datetime.now().isoformat()
    file_type, _ = mimetypes.guess_type(filename)

    metadata = {
        "filename": filename,
        "title": title[:100],
        "word_count": word_count,
        "keywords": keywords,
        "summary": semantic_summary,
        "language": language,
        "created_time": created_time,
        "file_type": file_type or "unknown"
    }

    return metadata


In [32]:
@app.route('/', methods=['GET'])
def home():
    return render_template('index.html')


In [37]:
@app.route('/', methods=['POST'])
def upload_file_2():
    file = request.files['file']
    if file:
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)
        return "File uploaded successfully"


In [39]:
@app.route('/', methods=['POST'])
def upload_file_v2():
    file = request.files['file']
    if file:
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)

        ext = filename.lower().split('.')[-1]
        if ext == 'pdf':
            text = extract_text_pdf(file_path)
        elif ext == 'docx':
            text = extract_text_docx(file_path)
        elif ext == 'txt':
            text = extract_text_txt(file_path)
        elif ext in ['png', 'jpg', 'jpeg']:
            text = extract_text_image(file_path)
        else:
            return "Unsupported file type."

        metadata = generate_metadata(text, filename)
        return jsonify(metadata)


In [40]:
text = "This is a sample document about data structures and memory management."
filename = "sample.txt"

metadata = generate_metadata(text, filename)
print(metadata)


{'filename': 'sample.txt', 'title': 'This is a sample document about data structures and memory management.', 'word_count': 11, 'keywords': ['sample', 'document', 'about', 'structures', 'memory', 'management'], 'summary': 'This is a sample document about data structures and memory management.', 'language': 'en', 'created_time': '2025-06-22T11:30:47.190682', 'file_type': 'text/plain'}


In [41]:
def save_metadata(filename, metadata):
    output_path = os.path.join(app.config['OUTPUT_FOLDER'], filename + '_metadata.json')
    with open(output_path, 'w', encoding='utf-8') as out_file:
        json.dump(metadata, out_file, indent=4)


In [42]:
import json
print(json.dumps(metadata, indent=4))


{
    "filename": "sample.txt",
    "title": "This is a sample document about data structures and memory management.",
    "word_count": 11,
    "keywords": [
        "sample",
        "document",
        "about",
        "structures",
        "memory",
        "management"
    ],
    "summary": "This is a sample document about data structures and memory management.",
    "language": "en",
    "created_time": "2025-06-22T11:30:47.190682",
    "file_type": "text/plain"
}


In [43]:
@app.route('/download/<filename>')
def download_metadata(filename):
    filepath = os.path.join(app.config['OUTPUT_FOLDER'], filename + '_metadata.json')
    return send_file(filepath, as_attachment=True)

In [ ]:
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
